# IBM GRANITE MODELS

- https://github.com/ibm-granite/granite-code-models

To use any of our models, pick an appropriate model_path from:

        ibm-granite/granite-3b-code-base

        ibm-granite/granite-3b-code-instruct

        ibm-granite/granite-8b-code-base

        ibm-granite/granite-8b-code-instruct

        ibm-granite/granite-20b-code-base

        ibm-granite/granite-20b-code-instruct

        ibm-granite/granite-34b-code-base

        ibm-granite/granite-34b-code-instruct

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, GenerationConfig
# model_path = "distilbert/distilgpt2" # pick anyone from above list
model_path = "ibm-granite/granite-3b-code-instruct" # pick anyone from above list
tokenizer = AutoTokenizer.from_pretrained(model_path)
# change input text as desired


In [2]:
# Loading the model - this will download the model to your local env. 
# https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration
model = AutoModelForCausalLM.from_pretrained(model_path)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at ibm-granite/granite-3b-code-instruct were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.11.mlp.down_proj.bias', 'model.layers.11.mlp.gate_proj.bias', 'model.layers.11.mlp.up_proj.bias', 'model.layers.12.mlp.down_proj.bias', 'model.layers.12.mlp.gate_proj.bias', 'model.layers.12.mlp.up_proj.bias', 'model.layers.13.mlp.down_proj.bias', 'model.layers.13.mlp.gate_proj.bias', 'model.layers.13.mlp.up_proj.bias', 'model.layers.14.mlp.down_proj.bias', 'model.layers.14.mlp.gate_proj.bias', 'model.layers.14.mlp.up_proj.bias', 'model.layers.15.mlp.down_proj.bias', 'model.layers.15.mlp.gate_proj.bias', 'model.layers.15.m

In [9]:
# Using chat 
# https://huggingface.co/ibm-granite/granite-8b-code-instruct

chat = [
    { "role": "user", "content": "Write a code to find the maximum value in a list of numbers." },
]
chat = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
# tokenize the text
input_tokens = tokenizer(chat, return_tensors="pt")
# transfer tokenized inputs to the device
for i in input_tokens:
    input_tokens[i] = input_tokens[i].to('cpu')


In [10]:
# generate output tokens
output = model.generate(**input_tokens, max_new_tokens=100)
# decode output tokens into text
output = tokenizer.batch_decode(output)
# loop over the batch to print, in this example the batch size is 1
for i in output:
    print(i)

In [ ]:
# With no chat
# https://huggingface.co/docs/transformers/main/en/main_classes/text_generation
input_text = '''Please generate the doc string section for this python function: def sum(a, b): retunrn a+b'''
# tokenize the text
input_tokens = tokenizer(input_text, return_tensors="pt")
for i in input_tokens:
    input_tokens[i] = input_tokens[i].to('cpu')

In [ ]:
output = model.generate(**input_tokens, max_new_tokens = 100)

In [ ]:
# decode output tokens into text
output = tokenizer.batch_decode(output)

# loop over the batch to print, in this example the batch size is 1
for i in output:
    print(i)

In [ ]:
# Original from Github above.
 
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
device = "cpu" #"cuda" # or "cpu"
model_path = "ibm-granite/granite-3b-code-instruct" # pick anyone from above list

tokenizer = AutoTokenizer.from_pretrained(model_path)
# Config added from repo to handle larger text inputs
config = AutoConfig.from_pretrained(model_path)
# config.max_seq_len = 4096
# config.max_answer_len= 1024
config.max_new_tokens = 250
config.max_length = 250

# drop device_map if running on CPU
model_code = AutoModelForCausalLM.from_pretrained(model_path
                                            #  , device_map=device
                                             , config = config)
model_code.eval()

In [ ]:
# change input text as desired
input_text = '''Please generate the doc string section for this python function: def sum(a, b): retunrn a+b'''
# tokenize the text
input_tokens = tokenizer(input_text, return_tensors="pt")
input_tokens

In [ ]:
# transfer tokenized inputs to the device
for i in input_tokens:
    input_tokens[i] = input_tokens[i].to(device)



In [ ]:
# generate output tokens
output = model_code.generate(**input_tokens)


In [ ]:
# decode output tokens into text
output = tokenizer.batch_decode(output)

# loop over the batch to print, in this example the batch size is 1
for i in output:
    print(i)

In [ ]:
def get_information(model, question, device):
    input_tokens = tokenizer(question, return_tensors="pt")

    # transfer tokenized inputs to the device
    for i in input_tokens:
         input_tokens[i] = input_tokens[i].to(device)

    # generate output tokens
    output = model.generate(**input_tokens)
    # decode output tokens into text
    output = tokenizer.batch_decode(output)
    return output

In [ ]:
test_text = '''Generte the docstring for the following puython function: 
def generate():
a = 1+1
return a'''
test_out = get_information(model_code, test_text, device)
# loop over the batch to print, in this example the batch size is 1
for i in test_out:
    print(i)

In [ ]:
base = '''what does this python function do {}'''
example1 = base.format('''
even= []
uneven=[]
for i in INPUT:
  if i %2 ==0:
    even.append(i)
  else:
    uneven.append(i)
org = uneven + even
print(org)''')
# loop over the batch to print, in this example the batch size is 1
for i in get_information(model_code, example1, device):
    print(i)


In [ ]:

headers= {
        "Authorization": f"Bearer {hug_face}",
        "Content-Type": "application/json"
        }

def hug_face_api_call(question, headers, API_URL=None):
    API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3" if API_URL == None else API_URL

    payload_template =  {
                    "inputs":question,
                    "parameters" : {
                                    "max_length": 40,
                                    }
                    }  
    try: 
        response = requests.post(API_URL, headers=headers, json=payload_template)
        answer = response.json()[0]['generated_text']
        return response, answer
    except Exception as E: 
        print(str(E))
        return

In [ ]:
example2 = base.format('''
final = []
for item in INPUT:
  if item %2 ==0:
    final.insert(len(final)+1, item)
  else:
    final.insert(0, item)
print(final)''')
# loop over the batch to print, in this example the batch size is 1
for i in get_information(model_code, example2, device):
    print(i)

In [ ]:
base = '''what does this python function do {}'''
example3 = base.format('''import pandas as pd
def create_df():
    data = {'state': ['Ohio','Ohio','Ohio','Nevada','Nevada'],
           'year': [2000,2001,2002,2001,2002],
           'pop': [1.5,1.7,3.6,2.4,2.9]}
    df = pd.DataFrame(data)
    return df

### We'll create three dataframes for an example
for i in range(3):
    exec(f'df_{i} = create_df()')''')
for i in get_information(model_code, example3, device):
    print(i)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # or "cpu"
model_path = "ibm-granite/granite-3b-code-base" # pick anyone from above list

tokenizer = AutoTokenizer.from_pretrained(model_path)

# drop device_map if running on CPU
model = AutoModelForCausalLM.from_pretrained(model_path)#, device_map=device)
model.eval()


In [ ]:

# change input text as desired
input_text = "def generate():"
# tokenize the text
input_tokens = tokenizer(input_text, return_tensors="pt")

# transfer tokenized inputs to the device
for i in input_tokens:
    input_tokens[i] = input_tokens[i]#.to(device)



In [ ]:
input_tokens

In [ ]:
# generate output tokens
output = model.generate(**input_tokens)


In [ ]:
output

In [ ]:
# decode output tokens into text
output = tokenizer.batch_decode(output)

# loop over the batch to print, in this example the batch size is 1
for i in output:
    print(i)